# Nantes: The West Coast of France

## An analysis of the neighbourhoods in the primary city of Nantes (excluding suburbs) with special emphasis on schools and top residential areas.

Problem:  There is very little useful information in English for new or would-be immigrants to Nantes about the different neighbourhoods, especially pertaining to early education and desirable residential areas.

Objective: To help new or would-be immigrants, who are not yet fluent in French, to find suitable schools for their children and accomodation in Nantes.
This report will provide information on :  
(1) schools (elementary vs. high school, public vs private, french vs bilingual/international),  
(2) detailed information such as venues of each neighbourhood,  
(3) areas to avoid,  
(4) average property prices of each neighbourhood, and  
(5) top areas to live. 

In [4]:
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import geocoder
import folium

print('Libraries imported.')

Libraries imported.


In [5]:
url = 'https://fr.wikipedia.org/wiki/Liste_des_quartiers_de_Nantes'

In [6]:
data  = requests.get(url).text 
soup = BS(data, 'html5lib')

In [7]:
table = soup.find('table')
for sup in table.find_all('sup'):
    sup.unwrap()
for span in table.find_all('span'):
    span.unwrap()
for a in table.find_all('a'):
    a.unwrap()

In [8]:
neighbourhoods = []
for row in table.find_all("tr"):
    if len(row.find_all('td')) >= 1:
        col = row.find_all('td')
        if (col != []):
            Neighbourhood = col[0].text
            Population = col[1].text
        neighbourhoods.append([Neighbourhood, Population])
neighbourhoods

[['Centre-ville[5]', '28 485'],
 ['Bellevue - Chantenay - Sainte-Anne[6]', '25 000'],
 ['Dervallières - Zola[7]', '35 000'],
 ['Hauts-Pavés - Saint-Félix[8]', '35 800'],
 ['Malakoff - Saint-Donatien[9]', '34 669'],
 ['Île de Nantes[10]', '15 818'],
 ['Breil - Barberie[11]', '24 418'],
 ['Nantes Nord[12]', '24 833'],
 ['Nantes Erdre[13]', '26 738'],
 ['Doulon - Bottière[14]', '30 147'],
 ['Nantes Sud[15]', '10 532']]

In [9]:
neighbourhoods1 = pd.DataFrame(neighbourhoods, columns = ['Neighbourhood', 'Population'])

In [10]:
neighbourhoods1

,Neighbourhood,Population
0,Centre-ville[5],28 485
1,Bellevue - Chantenay - Sainte-Anne[6],25 000
2,Dervallières - Zola[7],35 000
3,Hauts-Pavés - Saint-Félix[8],35 800
4,Malakoff - Saint-Donatien[9],34 669
5,Île de Nantes[10],15 818
6,Breil - Barberie[11],24 418
7,Nantes Nord[12],24 833
8,Nantes Erdre[13],26 738
9,Doulon - Bottière[14],30 147


In [11]:
import re
def purify(txt):
    return re.sub("\[.+\]", "", txt)

neighbourhoods1['Neighbourhood'] = neighbourhoods1['Neighbourhood'].apply(purify).copy()

In [12]:
neighbourhoods1

,Neighbourhood,Population
0,Centre-ville,28 485
1,Bellevue - Chantenay - Sainte-Anne,25 000
2,Dervallières - Zola,35 000
3,Hauts-Pavés - Saint-Félix,35 800
4,Malakoff - Saint-Donatien,34 669
5,Île de Nantes,15 818
6,Breil - Barberie,24 418
7,Nantes Nord,24 833
8,Nantes Erdre,26 738
9,Doulon - Bottière,30 147


In [13]:
coor = pd.read_csv('//Users//caroline//Desktop//nantes_quartiers_lat-long.csv')

In [14]:
coor.head()

,Quartier,Latitude,Longitude
0,Centre-ville,47.200142,-1.550053
1,Bellevue - Chantenay - Sainte-Anne,47.183478,-1.600042
2,Dervallières - Zola,47.216678,-1.583383
3,Hauts-Pavés - Saint-Félix,47.216794,-1.550108
4,Malakoff - Saint-Donatien,47.216736,-1.533361


In [15]:
coor = coor.rename(columns={'Quartier': 'Neighbourhood'})

In [16]:
coor.head()

,Neighbourhood,Latitude,Longitude
0,Centre-ville,47.200142,-1.550053
1,Bellevue - Chantenay - Sainte-Anne,47.183478,-1.600042
2,Dervallières - Zola,47.216678,-1.583383
3,Hauts-Pavés - Saint-Félix,47.216794,-1.550108
4,Malakoff - Saint-Donatien,47.216736,-1.533361


In [17]:
neighbourhoods_df = pd.merge(neighbourhoods1, coor)

In [18]:
neighbourhoods_df.head(13)

,Neighbourhood,Population,Latitude,Longitude
0,Centre-ville,28 485,47.200142,-1.550053
1,Bellevue - Chantenay - Sainte-Anne,25 000,47.183478,-1.600042
2,Dervallières - Zola,35 000,47.216678,-1.583383
3,Hauts-Pavés - Saint-Félix,35 800,47.216794,-1.550108
4,Malakoff - Saint-Donatien,34 669,47.216736,-1.533361
5,Île de Nantes,15 818,47.200042,-1.533472
6,Breil - Barberie,24 418,47.216831,-1.566789
7,Nantes Nord,24 833,47.250031,-1.550128
8,Nantes Erdre,26 738,47.250092,-1.516767
9,Doulon - Bottière,30 147,47.233394,-1.500089


In [19]:
neighbourhoods_df = pd.DataFrame(neighbourhoods_df)

In [20]:
from geopy.geocoders import Nominatim

In [21]:
address = 'Nantes, France'

geolocator = Nominatim(user_agent="nantes_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Nantes are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Nantes are 47.2186371, -1.5541362.


In [22]:
map_nantes= folium.Map(location=[latitude, longitude], zoom_start=12)

In [23]:
for lat, long, neighbourhood in zip(neighbourhoods_df["Latitude"], neighbourhoods_df["Longitude"], neighbourhoods_df["Neighbourhood"]):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nantes)  
    
map_nantes

In [24]:
# @hidden_cell
CLIENT_ID = 'D2JOFLEXDHMNPZGNMNIJR53L0ILI0UELERUPUB3XL4WREDS1'
CLIENT_SECRET = '0WFKC0ECIIPQ1C1M5OOCHONJBH1NZBTACNHDKCZT5KOUH0ES'
VERSION = '20180605'
radius = 5000
LIMIT = 100

categoryId =  
'4f4533804b9074f6e4fb0105' #elementary school  
'4bf58dd8d48988d13d941735' #high school  
'52e81612bcbc57f1066b7a46' #private school  
'52e81612bcbc57f1066b7a48' #language school  
'52e81612bcbc57f1066b7a47' #religious school  
'4bf58dd8d48988d13b941735' #school  
Nursery School
4f4533814b9074f6e4fb0107  
Preschool
52e81612bcbc57f1066b7a45

In [54]:
radius = 2000
limit = 100
for name, lat, lng in zip(neighbourhoods_df["Neighbourhood"], neighbourhoods_df["Latitude"], neighbourhoods_df["Longitude"]):
    print(name)
    
    categoryId = '4f4533804b9074f6e4fb0105'
         
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        lat,
        lng,
        categoryId,
        radius, 
        LIMIT)
       
    ES_resuls = requests.get(url).json()["response"]["venues"]
    ES_list= []
    for data in zip(ES_results):
        for d in data:
            E_name = d['name']
            E_lat = d['location']['lat']
            E_lng = d['location']['lng']
        ES_list.append([name, E_name, E_lat, E_lng])
    print(ES_list)

Centre-ville


TypeError: string indices must be integers

In [50]:
ES_df = pd.DataFrame(ES_results)
ES_df.head()
ES_df.shape

(4, 16)

In [99]:
radius = 2000
limit = 100
for name, lat, lng in zip(neighbourhoods_df["Neighbourhood"], neighbourhoods_df["Latitude"], neighbourhoods_df["Longitude"]):
    print(name)
    
    categoryId = '4f4533804b9074f6e4fb0105'
         
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        lat,
        lng,
        categoryId,
        radius, 
        LIMIT)
       
    ES_results = requests.get(url).json()['response']['venues']
    ES_list= []
    for data in zip(ES_results):
        for d in data:
            name = d['name']
            lat = d['location']['lat']
            lng = d['location']['lng']
        ES_list.append([name, lat, lng])
    print(ES_list)

Centre-ville
[['Ecole Emilie Pehant', 47.21171733197584, -1.5490886174653886], ['Ecole Elementaire Ledru Rollin', 47.192533880262346, -1.5344116997318822], ['École Bocage', 47.215716, -1.57043], ['École Louise Michel', 47.205307, -1.53103], ['Ecole Primaire Jean Jaures', 47.219422168533775, -1.5596448958186813], ['Ecole Saint Anne', 47.20284502383568, -1.5798874929762246], ['Ecole Elementaire Saint Nicolas', 47.21620515582818, -1.5628423242255631]]
Bellevue - Chantenay - Sainte-Anne
[['Ecole Primaire Du Fougan De Mer', 47.182608, -1.579806], ['Ecole primaire Alphonse Braud', 47.20095406159978, -1.5843266422146052], ['École primaire publique Les Reformes - Nantes', 47.20241173928157, -1.6032797440991424]]
Dervallières - Zola
[['Ecole Primaire Ampere', 47.2117805480957, -1.5846810340881348], ['École Du Soleil Levant', 47.219856, -1.610807], ['École maternelle Jacques Prévert', 47.2310637810822, -1.5771584660870175], ['École Bocage', 47.215716, -1.57043], ['Ecole Primaire Jean Jaures', 47

In [101]:
column_names = ['Neighbourhood', 'School Name', 'School Latitude', 'School Longitude']
ES_df = pd.DataFrame(columns = column_names)
for items in zip(ES_list):
    print(items)

(['École maternelle publique Le Douet', 47.191412083954624, -1.5191573458834695],)
(['École Sainte-Bernadette', 47.19280334684632, -1.5193163770973717],)
(['Ecole Elementaire Ledru Rollin', 47.192533880262346, -1.5344116997318822],)
(['École Henri Lesage', 47.18697825867775, -1.500406265258789],)


In [78]:
radius = 2000
limit = 100
for name, lat, lng in zip(neighbourhoods_df["Neighbourhood"], neighbourhoods_df["Latitude"], neighbourhoods_df["Longitude"]):
    print(name)
    
    categoryId = '4bf58dd8d48988d13d941735'
         
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        lat,
        lng,
        categoryId,
        radius, 
        LIMIT)
       
    HS_results = requests.get(url).json()['response']['venues']
    print(HS_results)

Centre-ville
[{'id': '50473f95e4b0f61f466141e2', 'name': 'Pôle des arts', 'location': {'lat': 47.205251969088835, 'lng': -1.5621783218762748, 'labeledLatLngs': [{'label': 'display', 'lat': 47.205251969088835, 'lng': -1.5621783218762748}], 'distance': 1079, 'cc': 'FR', 'country': 'France', 'formattedAddress': ['France']}, 'categories': [{'id': '4bf58dd8d48988d13d941735', 'name': 'High School', 'pluralName': 'High Schools', 'shortName': 'High School', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_', 'suffix': '.png'}, 'primary': True}], 'referralId': 'v-1624939885', 'hasPerk': False}, {'id': '4d4ba328b4b0a35d1b7213b4', 'name': 'Lycée Jean Perrin', 'location': {'address': '20 Rue Château de Rezé', 'lat': 47.181968828265965, 'lng': -1.5613245964050293, 'labeledLatLngs': [{'label': 'display', 'lat': 47.181968828265965, 'lng': -1.5613245964050293}], 'distance': 2195, 'postalCode': '44400', 'cc': 'FR', 'city': 'Rezé', 'state': 'Pays de la Loire', 'country': 'Franc

In [62]:
HS_list= []
for data in zip(HS_results):
    for d in data:
        h_name = d['name']
        h_lat = d['location']['lat']
        h_lng = d['location']['lng']
    HS_list.append([name, h_name, h_lat, h_lng])
print(HS_list)

[['Nantes Sud', 'Lycée Polyvalent Les Bourdonnières', 47.18603508747084, -1.5218819611617012], ['Nantes Sud', 'Lycée La Joliverie', 47.19049988963339, -1.5165424346923828]]


In [30]:
column_names = ['Neighbourhood', 'School Name', 'School Latitude', 'School Longitude']
HS_df = pd.DataFrame(columns = column_names)
for items in zip(HS_list):
    for item in items:
        HS_df = HS_df.append({'Neighbourhood': item[0],'School Name': item[1],
                      'School Latitude': item[2],'School Longitude': item[3]}, ignore_index=True)

HS_df.shape

(2, 4)

In [86]:
radius = 2000
limit = 100
for name, lat, lng in zip(neighbourhoods_df["Neighbourhood"], neighbourhoods_df["Latitude"], neighbourhoods_df["Longitude"]):
    print(name)
    
    categoryId = '4f4533814b9074f6e4fb0107'
         
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        lat,
        lng,
        categoryId,
        radius, 
        LIMIT)
       
    NS_results = requests.get(url).json()['response']['venues']
    NS_list= []
    for data in zip(NS_results):
        for d in data:
            name = d['name']
            lat = d['location']['lat']
            lng = d['location']['lng']
        NS_list.append([name, lat, lng])
    print(NS_list)

Centre-ville
[['Ecole Maternelle Sarah Bernard', 47.19281475347723, -1.5334441253006679], ['Ecole Joseph Blanchard', 47.21237587115002, -1.5647677779309757], ['Multi-Accueil Nouvelle Vague', 47.222302, -1.560678], ["gab's school", 47.19194761342661, -1.5688559422646968], ['Ecole La Martelliere', 47.198434182417245, -1.5251898616871533], ['École Publique maternelle Frédureau', 47.22096, -1.561459], ['Ecole Maternelle Henri Bergson', 47.217549195870866, -1.5289408134500144]]
Bellevue - Chantenay - Sainte-Anne
[['Ecole Maternelle Du Fougan De Mer', 47.183549, -1.578055]]
Dervallières - Zola
[['Ecole Joseph Blanchard', 47.21237587115002, -1.5647677779309757], ['Ecole Maternelle Dervalières Chézine', 47.22612380981445, -1.5972340106964111], ['Ecole De La Cremetterie', 47.21575880762606, -1.6083585133926344], ['Multi-Accueil Nouvelle Vague', 47.222302, -1.560678], ['École Publique maternelle Frédureau', 47.22096, -1.561459], ['Crèche Adelaïde', 47.225989, -1.560811], ['Ecole Georges Lafont',

In [90]:
np.ndarray.flatten(np.array(NS_list)).astype(object).tolist()
print(NS_list)

[['Ecole Sainte Famille', 47.18192244598738, -1.5108161057962333], ['Ecole La Martelliere', 47.198434182417245, -1.5251898616871533], ['Ecole Maternelle Sarah Bernard', 47.19281475347723, -1.5334441253006679]]


In [87]:
column_names = ['Neighbourhood', 'School Name', 'School Latitude', 'School Longitude']
NS_df = pd.DataFrame(columns = column_names)
for items in NS_list:
    for item in items:
        NS_df = NS_df.append({'Neighbourhood': item[0],'School Name': item[1],
                      'School Latitude': item[2],'School Longitude': item[3]}, ignore_index=True)

NS_df.shape

TypeError: 'float' object is not subscriptable

In [46]:
import requests
import json

query = "Internationale Bilingue"
cen_url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    47.2186371, 
    -1.5541362,
    query,
    radius, 
    LIMIT)
results = requests.get(cen_url).json()
results

{'meta': {'code': 200, 'requestId': '60da93d24798e163c51437a9'},
 'response': {'venues': [{'id': '4e70e5f8b0fba1302a65a185',
    'name': 'Bicloo Cité Internationale des Congrès N°54',
    'location': {'address': '2 rue de Jemmapes',
     'lat': 47.213218400618636,
     'lng': -1.544901095330715,
     'labeledLatLngs': [{'label': 'display',
       'lat': 47.213218400618636,
       'lng': -1.544901095330715}],
     'distance': 922,
     'postalCode': '44000',
     'cc': 'FR',
     'city': 'Nantes',
     'state': 'Pays de la Loire',
     'country': 'France',
     'formattedAddress': ['2 rue de Jemmapes', '44000 Nantes', 'France']},
    'categories': [{'id': '4e4c9077bd41f78e849722f9',
      'name': 'Bike Rental / Bike Share',
      'pluralName': 'Bike Rentals / Bike Shares',
      'shortName': 'Bike',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bikeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1624937426',
    'hasPerk': False},

In [146]:
prix = pd.read_csv('/Users/caroline/Desktop/BAROMETRE_PRIX_AVRIL_2021.csv')

In [126]:
prix.head()

,Quartier,Neuf,Ancien,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,NANTES,4 670€,3 650 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Centre-ville,6 150€,4 380 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chantenay - Bellevue - Sainte-Anne,4 750€,3 450 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dervallières - Zola,5 230€,3 840 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hauts Pavés - Saint Félix,6 150€,4 070 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
prix_df = prix[['Quartier', 'Neuf', 'Ancien']]

In [148]:
prix_df.head(13)

,Quartier,Neuf,Ancien
0,NANTES,4 670€,3 650 €
1,Centre-ville,6 150€,4 380 €
2,Chantenay - Bellevue - Sainte-Anne,4 750€,3 450 €
3,Dervallières - Zola,5 230€,3 840 €
4,Hauts Pavés - Saint Félix,6 150€,4 070 €
5,Saint Donatien - Malakoff,4 950€,3 770 €
6,Ile de Nantes,5 040€,3 390 €
7,Breil - Barberie,4 780 €,3 230 €
8,Quartiers Nord,4 720€,3 090 €
9,Nantes Erdre,3 980 €,3 270 €


In [149]:
prix_df1 = pd.DataFrame(prix_df)

In [150]:
prix_df2 = prix_df1.rename(columns={'Quartier': 'Neighbourhood', 'Neuf': 'New Homes (per m2)', 'Ancien': 'Old Homes (per m2)'})
prix_df2.head()

,Neighbourhood,New Homes (per m2),Old Homes (per m2)
0,NANTES,4 670€,3 650 €
1,Centre-ville,6 150€,4 380 €
2,Chantenay - Bellevue - Sainte-Anne,4 750€,3 450 €
3,Dervallières - Zola,5 230€,3 840 €
4,Hauts Pavés - Saint Félix,6 150€,4 070 €


In [151]:
prix_df2.loc[0,['Neighbourhood']] = 'Nantes'
prix_df2.loc[11,['Neighbourhood']] = 'Nantes Sud'
prix_df2.loc[4,['Neighbourhood']] = 'Hauts Paves - Saint Felix'
prix_df2.loc[6,['Neighbourhood']] = 'Ile de Nantes'
prix_df2.head(13)

,Neighbourhood,New Homes (per m2),Old Homes (per m2)
0,Nantes,4 670€,3 650 €
1,Centre-ville,6 150€,4 380 €
2,Chantenay - Bellevue - Sainte-Anne,4 750€,3 450 €
3,Dervallières - Zola,5 230€,3 840 €
4,Hauts Paves - Saint Felix,6 150€,4 070 €
5,Saint Donatien - Malakoff,4 950€,3 770 €
6,Ile de Nantes,5 040€,3 390 €
7,Breil - Barberie,4 780 €,3 230 €
8,Quartiers Nord,4 720€,3 090 €
9,Nantes Erdre,3 980 €,3 270 €


In [152]:
neighbourhoods_df.loc[-1] = ['Nantes', 0, 47.2186371, -1.5541362].copy()
neighbourhoods_df.index = neighbourhoods_df.index + 1
neighbourhoods_df = neighbourhoods_df.sort_index()
neighbourhoods_df.head(13)

,Neighbourhood,Population,Latitude,Longitude
0,Nantes,0,47.218637,-1.554136
1,Centre-ville,28 485,47.200142,-1.550053
2,Bellevue - Chantenay - Sainte-Anne,25 000,47.183478,-1.600042
3,Dervallières - Zola,35 000,47.216678,-1.583383
4,Hauts-Pavés - Saint-Félix,35 800,47.216794,-1.550108
5,Malakoff - Saint-Donatien,34 669,47.216736,-1.533361
6,Île de Nantes,15 818,47.200042,-1.533472
7,Breil - Barberie,24 418,47.216831,-1.566789
8,Nantes Nord,24 833,47.250031,-1.550128
9,Nantes Erdre,26 738,47.250092,-1.516767


In [153]:
neighbourhoods_df.loc[2,['Neighbourhood']] = 'Chantenay - Bellevue - Sainte-Anne'
neighbourhoods_df.loc[5,['Neighbourhood']] = 'Saint Donatien - Malakoff'
neighbourhoods_df.loc[8,['Neighbourhood']] = 'Quartiers Nord'
neighbourhoods_df.loc[4,['Neighbourhood']] = 'Hauts Paves - Saint Felix'
neighbourhoods_df.loc[6,['Neighbourhood']] = 'Ile de Nantes'
neighbourhoods_df.head(14)

,Neighbourhood,Population,Latitude,Longitude
0,Nantes,0,47.218637,-1.554136
1,Centre-ville,28 485,47.200142,-1.550053
2,Chantenay - Bellevue - Sainte-Anne,25 000,47.183478,-1.600042
3,Dervallières - Zola,35 000,47.216678,-1.583383
4,Hauts Paves - Saint Felix,35 800,47.216794,-1.550108
5,Saint Donatien - Malakoff,34 669,47.216736,-1.533361
6,Ile de Nantes,15 818,47.200042,-1.533472
7,Breil - Barberie,24 418,47.216831,-1.566789
8,Quartiers Nord,24 833,47.250031,-1.550128
9,Nantes Erdre,26 738,47.250092,-1.516767


In [154]:
neighbourhoods_df2 = pd.merge(neighbourhoods_df, prix_df2)
neighbourhoods_df2.head(13)

,Neighbourhood,Population,Latitude,Longitude,New Homes (per m2),Old Homes (per m2)
0,Nantes,0,47.218637,-1.554136,4 670€,3 650 €
1,Centre-ville,28 485,47.200142,-1.550053,6 150€,4 380 €
2,Chantenay - Bellevue - Sainte-Anne,25 000,47.183478,-1.600042,4 750€,3 450 €
3,Dervallières - Zola,35 000,47.216678,-1.583383,5 230€,3 840 €
4,Hauts Paves - Saint Felix,35 800,47.216794,-1.550108,6 150€,4 070 €
5,Saint Donatien - Malakoff,34 669,47.216736,-1.533361,4 950€,3 770 €
6,Ile de Nantes,15 818,47.200042,-1.533472,5 040€,3 390 €
7,Breil - Barberie,24 418,47.216831,-1.566789,4 780 €,3 230 €
8,Quartiers Nord,24 833,47.250031,-1.550128,4 720€,3 090 €
9,Nantes Erdre,26 738,47.250092,-1.516767,3 980 €,3 270 €
